In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## The dataset

The dataset `data/chip_plants.txt` is a table of chip manufacturing plants taken from [Wikipedia](https://en.wikipedia.org/wiki/List_of_semiconductor_fabrication_plants) and is in the [Wiki Markup](https://en.wikipedia.org/wiki/Help:Introduction_to_editing_with_Wiki_Markup/2) format. In this file, each cell starts with the character `|`, like this:

~~~plain
|[[Texas Instruments]]
~~~

Square brackets  (`[[]]`) represent links to Wikipedia pages. In the example above, the code directs you to [this page](https://en.wikipedia.org/wiki/Texas_Instruments).

Each cell is on its own line, and table rows are demarcated by this code: `|-`. Thus,

~~~plain
|-
|[[Texas Instruments]] (formerly [[Semiconductor Manufacturing International Corporation|SMIC]] - Cension)
|Chengdu (CFAB)
| China {{flagicon|China}}, Chengdu
|
|
|200
|
|
|
|-
|[[Tsinghua Holdings|Tsinghua Unigroup]]<ref name="eetasia.com"/>
|
| China {{flagicon|China}}, Nanjing
|10 (first phase), 30
|Planned
|300
|
|100,000 (first phase)
|3D NAND Flash
|-
~~~

produces this:

![Table example](data/images/table_example.png)

Finally, `<ref name="example.com"/>` adds a footnote with a hyperlink (see the example in the second row above) and `{{flagicon|country_name}}` adds a country flag.

The file has two columns - `line` is the actual Wikipedia cells and `id` is an identifier of the chip manufacturing plant.

## Import Data

In [ ]:
import pandas as pd

chips = pd.read_csv(r'/chip_plants.csv')
chips.head()


## Remove rows with extraneous information and blank spaces
---
[Series.str.strip()](https://docs.python.org/3/library/stdtypes.html#str.strip "Documentation") Return a copy of the string with the leading and trailing characters removed. The chars argument is a string specifying the set of characters to be removed. If omitted or None, the chars argument defaults to removing whitespace. 

In [ ]:
def remove_separators(chips):
    """
    Removes all rows that consist of the separator `|-`
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    `chips`: A pandas DataFrame (modified version of the input DataFrame)
    """
    
    # YOUR CODE HERE
    chips['input_column'] = chips['input_column'].str.strip()
    chips = chips[chips['input_column'] != 'string_to_remove']
    
    return chips


In [ ]:
# Call function here and set to a variable

[Series.str.replcace()](https://docs.python.org/3/library/stdtypes.html#str.replace "Documentation") Return a copy of the string with all occurrences of substring old replaced by new. If the optional argument count is given, only the first count occurrences are replaced.

[pandas.DataFrame.assign()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html "Documentation") Returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten

In [ ]:
def clean_cells(chips):
    """
    Removes unwanted characters (|,[,]) from the `line` column after having
    removed the rows that consist of the separator `|-`
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    `chips`: A pandas DataFrame (modified version of the input DataFrame)
    """
    chips = remove_separators(chips)
    
    # YOUR CODE HERE
    pd.set_option('mode.chained_assignment',None)
    remove_char = {'comma separated characters to remove'}
    for char in remove_char:
        chips['input_column'] = chips['input_column'].str.replace(char, '', regex=False)
    chips['input_column'] = chips['input_column'].str.strip()
    chips_line_clean = chips['input_column']
    chips_line_clean = chips_line_clean.str.replace(r"\<.*\>", "", regex=True)
    chips_line_clean = chips_line_clean.str.replace(r"\{.*\}", "", regex=True)
    chips = chips.assign(line=chips_line_clean)
    
    return chips
clean_cells(chips)

In [ ]:
# Call function here and set to a variable

## Recreate the original Wikipedia table

Each table row in the Wikipedia dataset has to have exactly 9 columns:

1. `company`
2. `plant_name`
3. `plant_location`
4. `plant_cost_us_billions`
5. `started_production`
6. `wafer_size`
7. `process_technology`
8. `production_capacity`
9. `technology_products`

In [ ]:
def assign_column_names(chips):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
       
    chips = clean_cells(chips)

    # YOUR CODE HERE
    col_names = []
    
    chips['columns'] = col_names*int((chips.shape[0]/len(col_names)))
    return chips


In [ ]:
# Call the function here

[pandas.pivot()](https://pandas.pydata.org/docs/reference/api/pandas.pivot.html "Documentation") Reshape data (produce a “pivot” table) based on column values. Uses unique values from specified index / columns to form axes of the resulting DataFrame. 

In [ ]:
# Use assign_column_names(df) with .pivot() to produce original table

## Creating a function to output our pivoted dataframe

In [ ]:
def make_df_wide(chips):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
    chips = assign_column_names(chips)
    
    # YOUR CODE HERE
    chips = chips.pivot(index='id', columns="columns",values= 'line')
    chips.to_csv(r'file_path/file_name',index=False)

### Checking for Missing Data

[glob.glob()](https://docs.python.org/3/library/glob.html "Documentation") Return a possibly-empty list of path names that match pathname, which must be a string containing a path specification.

[pandas.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html "Documentation") Concatenate pandas objects along a particular axis with optional set logic along the other axes.

In [ ]:
import pandas as pd
import glob



def concat_csv(path):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
    
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame


In [ ]:
# Set path string to a variable
path = r'/BigThinkFiles'
# Call function and set to a variable

[DataFrame.isnull()](https://docs.python.org/3/library/glob.html "Documentation") Detect missing values.

[DataFrame.sum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html "Documentation") Return the sum of the values over the requested axis.

[DataFrame.sort_values()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html "Documentation") Sort by the values along either axis.

In [ ]:
def missing_data(frame):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
    
    percent_missing = frame.isnull().sum() * 100 / len(frame)
    missing_value_df = pd.DataFrame({'column_name': frame.columns,
                                    'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', inplace=True)

    return missing_value_df

In [ ]:
# Apply missing_data(df) to the concatinated dataframe

## Import "listing_complaints.csv"
---
This data set contains counts of air quality complaints from resident in NYC per Borough, UHF 42 district, and year


In [ ]:
airQ_complaints = pd.read_csv(r'/listing_complaints.csv')

In [ ]:
# Apply missing_data(df) to the listing_complaints
